<h1>contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#setup" data-toc-modified-id="setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#data" data-toc-modified-id="data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>data</a></span></li></ul></div>

# setup

In [15]:
import datetime
import gc
import json
import os
import sys
import pprint
import warnings
from io import BytesIO

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import folium
import geopandas as gpd
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import netCDF4 as nc
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from folium.plugins import HeatMap
from IPython.display import display
from scipy.interpolate import griddata
from selenium import webdriver
from skimage.transform import resize
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.preprocessing import MinMaxScaler

mpl.rcParams.update(mpl.rcParamsDefault)
warnings.filterwarnings("ignore")

%matplotlib inline

In [16]:
sns.set_style("whitegrid")

plt.rcParams.update({
    'font.size'       : 9,
    'figure.figsize'  : (3.5, 2.8),
    'figure.dpi'      : 300,
    'savefig.dpi'     : 300,
    'text.usetex'     : True,
    'font.family'     : 'serif',
    'font.serif'      : ['Times New Roman'],
    'axes.labelsize'  : 9,
    'axes.titlesize'  : 9,
    'xtick.labelsize' : 8,
    'ytick.labelsize' : 8,
    'legend.fontsize' : 8,
    'lines.linewidth' : 1,
    'axes.linewidth'  : 1,
    'grid.linestyle'  : '--',
    'grid.linewidth'  : 0.5,
    'grid.alpha'      : 0.8,
})

sns.set_context(
    "paper", 
    rc={
        "font.size"      : 9,
        "axes.titlesize" : 9,
        "axes.labelsize" : 9, 
        'xtick.labelsize': 8,
        'ytick.labelsize': 8
    }
)

In [17]:
experiment_name = "wildfire_spread"
folder_path = f"../data/08_reporting/{experiment_name}"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

golden_ratio = (np.sqrt(5) - 1) / 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
BRAZIL_EXTENT = [-74, -34, -34, 6]
SAO_PAULO_EXTENT = [-53, -44, -25, -20]

# data

In [5]:
MAX_LAT = 32.99499999641088
MIN_LAT = -55.98500000358912
MAX_LON = -33.009801806103724
MIN_LON = -119.98980180610373

MAX_LAT = BRAZIL_EXTENT[3]
MIN_LAT = BRAZIL_EXTENT[2]
MAX_LON = BRAZIL_EXTENT[1]
MIN_LON = BRAZIL_EXTENT[0]

loading npz files

In [6]:
TENSOR = torch.empty((0, 6, 3000, 3000), dtype=torch.float16)

for _dt in pd.date_range("2023-10-31", "2023-11-19", freq="D"):
    try:
        print(_dt.strftime("%Y-%m-%d"))
        _data = np.load(
            f"../data/02_intermediate/wildfire_spread/{_dt.strftime('%Y%m%d')}.npz",
        )
        _timestamp = _data["timestamps"]
        _tensor = _data["tensor"].astype(np.float16)
        _tensor = torch.from_numpy(_tensor)
        _tensor = _tensor.unsqueeze(0)
        TENSOR = torch.cat((TENSOR, _tensor), dim=0)
        del _tensor
        del _data
        gc.collect()
    except Exception as e:
        print(e)
        continue

2023-10-31
2023-11-01
2023-11-02
2023-11-03
2023-11-04
2023-11-05
2023-11-06
[Errno 2] No such file or directory: '../data/02_intermediate/wildfire_spread/20231106.npz'
2023-11-07
[Errno 2] No such file or directory: '../data/02_intermediate/wildfire_spread/20231107.npz'
2023-11-08
[Errno 2] No such file or directory: '../data/02_intermediate/wildfire_spread/20231108.npz'
2023-11-09
2023-11-10
2023-11-11
2023-11-12
2023-11-13
2023-11-14
2023-11-15
2023-11-16
2023-11-17
2023-11-18
2023-11-19
[Errno 2] No such file or directory: '../data/02_intermediate/wildfire_spread/20231119.npz'


In [7]:
# saving
torch.save(TENSOR, f"../data/02_intermediate/wildfire_spread/wildfire_spread_tensor.pt")

In [20]:
def update_plot(frame, layer_data, img):
    img.set_data(layer_data[frame])
    return img,

In [26]:
LABELS = {
    0 : "Wildifire",
    1 : "Days Without Rain",
    2 : "Precipitation",
    3 : "Temperature",
    4 : "Relative Humidity",
    5 : "Fire Risk",
}

for _t, _timestamp in enumerate(TENSOR):
    for _index, _layer in enumerate(_timestamp):
        # plot each of 6 layers using cartopy
        fig = plt.figure(figsize=(5, 5))
        ax = plt.axes(projection=ccrs.PlateCarree(), frameon=False)
        ax.set_extent([MIN_LON, MAX_LON, MIN_LAT, MAX_LAT], crs=ccrs.PlateCarree())

        ax.imshow(
            _layer,
            origin="upper",
            extent=[MIN_LON, MAX_LON, MIN_LAT, MAX_LAT],
            transform=ccrs.PlateCarree(),
            cmap="gray"
        )
        
        plt.title(LABELS[_index])
        plt.tight_layout()
        plt.savefig(f"../data/08_reporting/wildfire_spread/{LABELS[_index].replace(' ', '-')}_{_t}.png".lower(), dpi=300)

        plt.close(fig)

auxiliary functions

In [15]:
def sizeof_fmt(num, suffix='B'):
    """
    Convert size of objects in memory
    """
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "{:>6.1f} {}{}".format(num, unit, suffix)
        num /= 1024.0
    return "{:.1f} {}{}".format(num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                       HeatMap:    1.0 KiB
                 ParameterGrid:    1.0 KiB
                  MinMaxScaler:    1.0 KiB
                           _i1:  947.0 B
                           _i2:  806.0 B
                          _i10:  752.0 B
                           _i9:  655.0 B
                           _i6:  613.0 B
                          _iii:  536.0 B
                          _i12:  536.0 B


In [16]:
del __

In [13]:
gc.collect()

0